# Prototype: Verification

In [1]:
import open3d as o3d
import numpy as np

import torch
import torch.nn as nn
from torch.autograd import Variable
import open3d.visualization as vis

import sys
sys.path.insert(0, '../impl/utils/')
sys.path.insert(0, '../')
import voxel_processing as vp

# device will determine whether to run the training on GPU or CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


device(type='cuda')

In [2]:
from impl.PRSNet import PRSNet_Loss
criterion = PRSNet_Loss(25)
criterion.to(device)

PRSNet_Loss(
  (symmetry_loss): PRSNet_Symm_Dist_Loss()
  (reg_loss): PRSNet_Reg_Loss()
)

In [24]:
mesh = vp.read_mesh_from_file("../data/func_test/voxel/prs_dat_0.objdat")
mesh.compute_vertex_normals()
plane = np.array([0.5708, 0.0789, 0.0323, 0.0152], dtype=np.float32)
import visualization as prsvis
prsvis.visualize_planar_symmetry(mesh, plane)

[visualization.py] Drawing symmetry plane [0.5708 0.0789 0.0323 0.0152]


In [30]:
index, omap, grid_points, sample_points = vp.prepare_single_data(('../data/func_test/voxel/prs_dat_0.voxdat', 1))

Test if transformed mesh is within bounds

In [31]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(sample_points)
frame = o3d.geometry.TriangleMesh.create_coordinate_frame()
o3d.visualization.draw_geometries([pcd, mesh, frame])

In [34]:
pl = torch.tensor([0.75011107,  0.         ,-0.66131187, 0], device=device, dtype=torch.float32)
planar_features = pl.reshape(1, 1, 4).repeat(1, 3, 1)

SP_train = sample_points.reshape(1, 1000, 3).to(device, dtype=torch.float32)
GP_train = grid_points.reshape(1, 32, 32, 32, 3).to(device, dtype=torch.float32)
SP_trans = criterion.symmetry_loss.apply_planar_transform(planar_features, SP_train)
criterion.symmetry_loss.compute_batch_dist_sum(GP_train, SP_trans.reshape(1, -1, 3)).sum()

tensor(148.7330, device='cuda:0')

Show Transformed Points

In [70]:
pcd1 = o3d.geometry.PointCloud()
arr = np.array(SP_trans.detach().cpu().numpy())
pcd1.points = o3d.utility.Vector3dVector(arr[:, :1000, 0, :].reshape(-1, 3))
frame = o3d.geometry.TriangleMesh.create_coordinate_frame()
o3d.visualization.draw_geometries([pcd1, transformed_mesh, frame])

Show Grid Points

In [71]:
pcd2 = o3d.geometry.PointCloud()
arr = np.array(GP_train.detach().cpu().numpy())
pcd2.points = o3d.utility.Vector3dVector(arr.reshape(-1, 3))
frame = o3d.geometry.TriangleMesh.create_coordinate_frame()
o3d.visualization.draw_geometries([pcd2, mesh, frame])

In [ ]:
pcd2 = o3d.geometry.PointCloud()
pcd2.points = o3d.utility.Vector3dVector(np.array(SP_trans[:, :, 0].reshape(-1, 3).contiguous().cpu()))
frame = o3d.geometry.TriangleMesh.create_coordinate_frame()
o3d.visualization.draw_geometries([pcd2, mesh, frame])